# Identificación de Retinopatía Diabética con Arquitectura de Aswin Shriram Thiagarajan et al

Paper: Aswin Shriram Thiagarajan et al., / Journal of Computer Science 2020, 16 (3): 305.313 

__DOI: 10.3844/jcssp.2020.305.313__

In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from imutils import paths

from tesis_lib.nn.thiagarajan import Thiagarajan
from tesis_lib.io.hdf5datasetwriter import HDF5DatasetWriter
from tesis_lib.io.hdf5datasetgenerator import HDF5DatasetGenerator
from tesis_lib.datasets.rosenbrock_loader import RosenbrockLoader
from tesis_lib.callbacks.trainingmonitor import TrainingMonitor
from tesis_lib.preprocessing.imagetoarrayprocessor import ImageToArrayPreprocessor
from tesis_lib.preprocessing.aspectawareprocessor import AspectAwareProcessor
from tesis_lib.preprocessing.patchpreprocessor import PatchPreprocessor
from tesis_lib.preprocessing.meanpreprocessor import MeanPreprocessor
from tesis_lib.preprocessing.simplepreprocessor import SimpleProcessor

from keras import backend as K
from keras.callbacks import LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam


import numpy as np
import progressbar
import json
import cv2
import os

2023-01-24 12:17:05.177595: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from datetime import datetime

DATASET_PATH = './DB'
SPLIT_PATHS = ['Training', 'Validation', 'Test']

MODEL_OUT_PATH = f'./models/thiagarajan_{datetime.today().strftime("d_m_Y_H_M_s")}.h5'
EPOCHS = 20
IM_SIZE = 256
NUM_CLASSES = 2

In [3]:
aap = AspectAwareProcessor(IM_SIZE,IM_SIZE)
iap = ImageToArrayPreprocessor()
(R,G,B) = ([],[],[])


for dType in SPLIT_PATHS:
    path = os.path.sep.join([DATASET_PATH, dType])

    imagePaths = [list(paths.list_images(DATASET_PATH))
        for im_class in os.listdir(path)][0]

    labels = [pt.split(os.path.sep)[-2] for pt in imagePaths]

    le = LabelEncoder()
    labels = le.fit_transform(labels)

    # Dataset[s]["im_paths"] = imagePaths
    # Dataset[s]["labels"] = labels

    # display(Dataset[s])
    if os.path.exists(f'./DB/hdf5/{dType}.hdf5'):
      os.remove(f'./DB/hdf5/{dType}.hdf5')
    writer = HDF5DatasetWriter((len(imagePaths), IM_SIZE,IM_SIZE,3), f'./DB/hdf5/{dType}.hdf5')

    widgets = [
        f"Building Dataset {dType}: ",
        progressbar.Percentage(),
        " ",
        progressbar.Bar(),
        " ",
        progressbar.ETA()
    ]

    pbar = progressbar.ProgressBar(
        maxval=len(imagePaths),
        widgets=widgets
        ).start()
    
    for (i, (path,label)) in enumerate(zip(imagePaths, labels)):
        image = cv2.imread(path)
        try:
          image = aap.preprocess(image)
        except Exception:
          display(f"[WARNING] Skipped {path.split('/')[-1]}")
          continue

        if dType == "Training":
            (b,g,r) = cv2.mean(image)[:3]
            R.append(r)
            G.append(g)
            B.append(b)
        
        writer.add([image], [label])
        pbar.update(i)
    
    pbar.finish()
    writer.close()

Building Dataset Training: 100% |###############################| Time: 0:04:16
Building Dataset Validation: 100% |#############################| Time: 0:04:06
Building Dataset Test: 100% |###################################| Time: 0:04:02


In [4]:
print("[INFO] serializing means...")
D = {
    "R": np.mean(R),
    "G": np.mean(G),
    "B": np.mean(B)
}
with open('./DB/hdf5/diat_ret.json', "w") as f:
    f.write(json.dumps(D))

[INFO] serializing means...


In [3]:
aug = ImageDataGenerator(
    rotation_range=25, 
    shear_range=0.20, 
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1, 
    horizontal_flip=True, 
    fill_mode='nearest')

In [4]:
means = json.loads(open('./DB/hdf5/diat_ret.json').read())

sp = SimpleProcessor(IM_SIZE, IM_SIZE)
pp = PatchPreprocessor(IM_SIZE,IM_SIZE)
mp = MeanPreprocessor(means["R"], means["G"], means["B"])
iap = ImageToArrayPreprocessor()

In [5]:
trainGen = HDF5DatasetGenerator(
    'DB/hdf5/Training.hdf5',
    128,
    aug=aug,
    preprocessors=[pp, mp, iap],
    classes=NUM_CLASSES
)

valGen = HDF5DatasetGenerator(
    'DB/hdf5/Validation.hdf5',
    128,
    aug=aug,
    preprocessors=[sp, mp, iap],
    classes=NUM_CLASSES
)

In [1]:
from tensorflow.python.client import device_lib
display(device_lib.list_local_devices())

2023-01-24 13:39:18.664236: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-24 13:39:20.372668: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9827087165459442062
 xla_global_id: -1]

In [8]:
print("[INFO] Compiling model ...")

opt = Adam(lr=1e-3)

model = Thiagarajan.build(
    width= IM_SIZE,
    height= IM_SIZE,
    depth= 3,
    classes= NUM_CLASSES
)

model.compile(
    loss="binary_crossentropy",
    optimizer=opt,
    metrics=["accuracy"]
)

monitor_path = os.path.sep.join([
    './output/',
    f"{os.getpid()}.jpg"
])

callbacks = [TrainingMonitor(monitor_path)]

print(f"[INFO] training model ...")

H = model.fit(
    trainGen.generator(),
    steps_per_epoch=trainGen.numImages // 128,
    validation_data=valGen.generator(),
    validation_steps=valGen.numImages // 128,
    epochs= EPOCHS,
    max_queue_size=10,
    callbacks=callbacks,
    verbose=1
)


print("[INFO] serializing model ...")
model.save(MODEL_OUT_PATH, overwrite=True)

[INFO] Compiling model ...


/home/newton/miniconda3/envs/expmlenv/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2023-01-24 11:48:17.579860: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-24 11:48:17.610590: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


[INFO] training model ...
Epoch 1/20
25/25 [==============================] - 130s 5s/step - loss: 0.8816 - accuracy: 0.5503 - val_loss: 0.8972 - val_accuracy: 0.3594
Epoch 2/20
25/25 [==============================] - 132s 5s/step - loss: 0.6799 - accuracy: 0.6803 - val_loss: 0.7049 - val_accuracy: 0.6981
Epoch 3/20
25/25 [==============================] - 137s 6s/step - loss: 0.6189 - accuracy: 0.7406 - val_loss: 0.8644 - val_accuracy: 0.5234
Epoch 4/20
17/25 [===================>..........] - ETA: 31s - loss: 0.6283 - accuracy: 0.7422

KeyboardInterrupt: 